In [1]:
from pathlib import Path
import sys

def encontrar_raiz_proyecto(nombre_carpeta='src'):
    """Busca la raíz subiendo niveles hasta encontrar la carpeta src"""
    current = Path.cwd()
    
    # Busca en directorios padre
    for path in [current] + list(current.parents):
        if (path / nombre_carpeta).exists():
            return path
    
    raise FileNotFoundError(f"No se encontró la carpeta '{nombre_carpeta}'. ¿Estás dentro del proyecto?")

# Uso
project_root = encontrar_raiz_proyecto('src')
sys.path.insert(0, str(project_root))

# Ahora importas normal
from src.Obtener_Datos import Abrir_archivos, Data, Modificar_nombre, Leer_archivo_Excel, Comparador_Fuzzy

In [2]:
from pathlib import Path
import glob

# Ruta del notebook actual -> sube un nivel -> entra a EQUIPOS PANAMA
carpeta_equipos = Path.cwd().parent / "EQUIPOS PANAMA"

# Verificar que existe (buena práctica)
if not carpeta_equipos.exists():
    raise FileNotFoundError(f"No se encuentra la carpeta: {carpeta_equipos}")

# Buscar archivos CSV
archivos = list(carpeta_equipos.glob("*.csv"))
print(f'Se encontraron {len(archivos)} archivos CSV en: {carpeta_equipos}')

DATOS_ORIGINALES = Data.procesar_Data(archivos)
DATOS_MODIFICADOS = Modificar_nombre.cambio_nombres(DATOS_ORIGINALES)

Se encontraron 54 archivos CSV en: c:\Users\monitoreouio2\Desktop\Proyecto Base Datos\EQUIPOS PANAMA


In [3]:
ruta_excel = [r"C:\Users\monitoreouio2\Desktop\Proyecto Base Datos\DATOS EL REY.xlsx"]
#ruta_excel = Abrir_archivos.seleccionar_archivos()
df = Leer_archivo_Excel.leer_excel_a_dataframe(ruta_excel, "Hoja1")

DATOS_ESTRUCTURADOS = Comparador_Fuzzy.procesar_similitudes(df,DATOS_MODIFICADOS,usar_fuzzy=True)

In [4]:
#DATOS_ESTRUCTURADOS
#df

In [5]:
import pandas as pd

df_estructurado = []
for locales in DATOS_ESTRUCTURADOS:
    df_estructurado.append(locales[0])
df_estructurado = pd.DataFrame(df_estructurado)
titulos = df.columns
df_estructurado.columns = [titulos[0], titulos[1], titulos[2]]


# Solo en df1 (no están en df2)
solo_en_df1 = df.merge(df_estructurado, how='outer', indicator=True).loc[lambda x: x['_merge'] == 'left_only']
solo_en_df1



,REGION,FORMATO,NOMBRE LOCAL,_merge
6,R1,MR PRECIO,SANTA FE,left_only
9,R1,REY,BRISAS NORTE,left_only
22,R2,METROPLUS,VÍA PORRAS,left_only
27,R3,METROPLUS,ALTOS DE PANAMÁ,left_only
34,R3,METROPLUS,VALLARINO,left_only
44,R4,MR PRECIO,VISTA MAR,left_only
46,R4,REY,CALLE 13,left_only
47,R4,REY,CALLE 7,left_only
54,R5,MR PRECIO,LAS TABLAS,left_only
59,R5,ROMERO,PLAZA BUGABA,left_only
